<div class="alert alert-block alert-warning">
<b>Warning:</b>
 To successfuly run these analyses you should run the following notebooks before:
</div>

- [preprocessing](http://nbviewer.jupyter.org/urls/bitbucket.org/bbglab/mutfootprints/raw/master/preprocessing_data.ipynb)

- [formating and annotating](http://nbviewer.jupyter.org/urls/bitbucket.org/bbglab/mutfootprints/raw/master/formatting_and_annotating.ipynb)

# Create the matrix input for signature extraction

We should run the `script src/signatures/prepare_matrix_extraction.py` for each of the merged cohorts. This will generate matrices for snvs, indels and dbs. Specifically, it generates;

- 1536 channels (COMPOSITE in PCAWG Signatures publication)
- 96 SNV
- 86 indels
- 72 DBS

Then it generates the same but taking into account replication and transcription (thus doubling the amount of channels)

The script can be run as follow:
```Python
python src/signatures/prepare_matrix_extraction.py -c <MERGED_COHORT> -o <OUTPUT_DIRECTORY>
```


# Download PCAWG canonical signatures 

The set of PCAWG signatures and its postprocessing are already included in the repository. Nevertheless, if you want to reproduce what we did in the paper, follow these instructions.

The set of signatures can be obtained from [Synapse](https://www.synapse.org/#!Synapse:syn12025148).
Our set was the version from `_2018_03_28`. 

Then run 
```
python src/signatures/sign_utils/sort_index_signatures.py
```

to postprocess some of the files and make them compatible for downstream analyses

# Run both extraction methods

## SigProfilerJulia

SigProfiler_Julia can be run as follows:

```Bash
$PATH_TO_JULIA/julia $PATH_TO_SIGPROFILER/SigProfilerJulia.jl -f data/hartwig/signatures/matrix/PanNoSkinNoUnknown.dbs.dlm -s 2 -m 7 -i 1024 --workers 400 --outpath data/signature_extraction/SigProfiler/ --slurm

```
Where \$PATH_TO_JULIA is the location of the julia binary, and \$PATH_TO_SIGPROFILER is where SigProfilerJulia has been cloned. Please see [SigProfilerJulia](https://bitbucket.org/bbglab/sigprofilerjulia/) repository for more details.

---


## SignatureAnalyzer

This script can be run as follows : 

```R
Rscript src/signatures/signature_analyzer_sender.R data/signatures/PanNotHypermutated_full.snvs.dlm 70 data/signature_extraction/SignatureAnalyzer/ PAN
```

Where the first parameter is the matrix, the second is the maximum number of signatures allowed, the third is the outpath and the last one defines the parameter "tolerance" (with PAN meaning more relaxed). Please be aware that if "NotHypermutated" is found in the file name, the script will try to locate the Hypermutated file too and do a similar processing as explained in the PCAWG publication.
Also please note that **the script will try to use all the CPUs allocated to your job.**

---


# Postprocessing

The postprocessing of the signatures calculated by SigProfiler and SygnatureAnalyzer is done by the following scripts:

In [ ]:
%%bash

#SignatureAnalyzer
python src/signatures/signatureanalyzer_postprocess.py \
-i 'data/hartwig/signatures/extraction/SignatureAnalyzer/' \
-o 'data/hartwig/signatures/extraction/results/SignatureAnalyzer/'

In [ ]:
%%bash

#SigProfiler
python src/signatures/sigprofiler_postprocess.py

The above commands will generate the mutational spectra for each of the signatures extracted, and will locate all the results in ```data/hartwig/signatures/extraction/results/ by default```

# Other figures

In [ ]:
%%bash

#Supplementary Note Figure 2
python src/signatures/sign_utils/plot_dots_signatures.py

#Extended Data Figure 2
python src/signatures/sign_utils/asymmetry_signature_analysis.py

# Hierarchical Dirichlet Process extraction

In [ ]:
%%bash

bash src/signatures/sign_utils/HDP_pipeline.sh